# Python code for final project PHZ5305: Interaction matrix elements and Hartree-Fock

## Introduction.

This code is written for the final project of PHZ5305 (Nuclear Physic 1), which will provide the tool for reads the list of single-particle states and the me2b generated, and solves the Hartree-Fock Equation. 


## Import necessary packages.

`numpy` is imported for computational works. `pandas` is imported for reading the values of me2b from outer sources, and `itertools` for iterational job for lists, for example permutations. 

In [1]:
import numpy as np
import pandas as pd
import itertools as itr
import sympy as sp
from sympy.physics.quantum.cg import CG
from sympy import S
from decimal import Decimal
import random

## Define Class for Single-Particle States

In this block, the class for single-particles states are generated.

In the `__init__` block, the basic definition of `SP_States` are implied.

In the `nucleon_loop` block, the loop that generates `SP_list` is encoded.

In the `generate_lists` block, by using `nucleon_loop` function twice to make the list of single particle states. 

In [8]:
class SP_States:
    # Initialize the function
    def __init__(self,A,Z):
        self.A = A                                     # Atomic Mass
        self.Z = Z                                     # Number of Protons
        self.N = A-Z                                   # Number of Neutrons
        self.i_degen = [2, 4, 2]                       # degeneracy for 0s1/2, 0p3/2, 0p1/2 shells
        self.i_numsh = [0, 1, 1]                       # Will be implied as l
    
    # Loop for each nucleon that generates (index, n, l, j, mj, tz) lists for each single particle states
    def nucleon_loop(self, SP_list, tz):
        i_shell = 0
        N_nucl  = 0                                    # Number of Nucleon
        I_nucl  = 0 
        
        if   tz == -0.5 : N_nucl = 8 # test: original value was self.Z
        elif tz ==  0.5 : N_nucl = 8 # test: original value was self.N
        
        index = len(SP_list)
        
        for itr in range(len(self.i_degen)):
            n     = 0                                  # shell's n
            l     = self.i_numsh[itr]                  # for first shell, l = 0, and for second and third shell, l = 1
            j     = (self.i_degen[itr]-1)/2            # from num. deg. states = 2J+1 -> J = (num. deg. states-1)/2 
            mj    = -j                                 # Starting from mj = -j
            if I_nucl == N_nucl: break                 # Break the loop when it reaches N_nucl
            while mj < j+1 :                           # Start iteration from mj = -j to mj = +j
                SP_list.append([index,n,l,j,mj,tz])    # Generate the single-particle states for protons
                mj     = mj     + 1                    # mj up
                I_nucl = I_nucl + 1
                index  = index  + 1                    # index   

        else: i_shell = i_shell+1
    
    # Function generates single particle's list
    def generate_lists_SP(self):
        SP_list = []                                   
        
        self.nucleon_loop(SP_list, -0.5)               # Loop for Proton
        self.nucleon_loop(SP_list,  0.5)               # Loop for Neutron
        
        self.SP_list = SP_list
        return SP_list
        

## Define class for generate two-particle states from one-particle states, and antisymmetrize it.

In this block, the class that have an input of the list of single-particle states and generate the list of double-particle states.

`generate_lists_TB` function generates the full lists of two-body state's indices.

`assymetrizer` function asymmetrize the two-body states in the two-body states lists, and return the asymmetrized two-body states.

`TB_analyzer` function gives a cut for two-body states to be coupled with Jpi=0+

In [9]:
class TB_States:
    # Initial definition of Two-body states
    def __init__(self,SP_States):
        self.A = SP_States.A                             # Atomic mass
        self.Z = SP_States.Z                             # Number of Protons
        self.N = SP_States.N                             # Number of Neutrons
        self.list_SP = SP_States.generate_lists_SP()     # Generate the Single Particle States
        
    # Function that generates the list of two-body states, not antisymmetrized yet.
    def generate_lists_TB(self):                         
        Ind_SP = list(np.arange(0,len(self.list_SP),1))  
        Ind_TB = [[i_SP, j_SP] for i_SP in Ind_SP for j_SP in Ind_SP if i_SP<j_SP and self.TB_analyzer([i_SP,j_SP])] 
                                                         
        self.Ind_TB = Ind_TB
        return Ind_TB
        
    # Function that assymetrize the result of generate_lists_TB
    def assymetrizer(self):                              
        self.generate_lists_TB()                         
        Asym_Ind_TB = [list(itr.permutations(i_Ind_TB)) for i_Ind_TB in self.Ind_TB if self.TB_analyzer(i_Ind_TB)]
                                                         # Generate all the permutations in Ind_TB
        self.Asym_Ind_TB = Asym_Ind_TB
        return Asym_Ind_TB                               
        
    # Input two-body state (a,b) and analyize the state is in Jp=0+
    def TB_analyzer(self, TB_State):                     
        Stat1 = self.list_SP[int(TB_State[0])]           
        Stat2 = self.list_SP[int(TB_State[1])]           
        
        n1, l1, j1, mj1 = Stat1[1], Stat1[2], Stat1[3], Stat1[4]
        n2, l2, j2, mj2 = Stat2[1], Stat2[2], Stat2[3], Stat2[4]
        
        #return True
        
        if l1 == l2 and mj1 == -1*mj2 and j1 == j2 and n1==n2 : return True
        else: return False
        
        
        

## Define class for Hamiltonian and Solve Hartree-Fock.

In this block, the Hartree-Fock equation is solved via algorithm that introduced at the class (Oct-14-2024 class), with auto-generated Hamiltonian

`generate_HFMatrix` is the most important block, which generates Coefficient matrix and density matrix, and by using them, generates the energy states by solving the Hartree-Fock equations.

`onebody` is a function for generate the one-body Hamiltonian

`twobody` is a function for generate the two-body matrix elements (me2b), but it should be filled (Nov 27 2024 comment)

`krondelt` is a function for the Kronecker-Delta, just added for future purpose currently (Nov 27 2024 comment)


In [12]:
class Hamiltonian:
    # Initial definition of states
    def __init__(self, SP_States):
        self.A = SP_States.A                                # Atomic mass
        self.Z = SP_States.Z                                # Number of Protons
        self.N = SP_States.N                                # Number of Neutrons
        self.list_SP = SP_States.generate_lists_SP()        # Generate the Single Particle States
        self.list_TB = TB_States(SP_States).assymetrizer()  # Generate the Two-Particle States
        
    #Solving the Hartree-Fock Equation
    def generate_HFMatrix(self):
        Nind = len(self.list_SP)
        
        # Computing single-particle Hamiltonian (SPH)
        SPH = np.array([self.onebody(sp) for sp in self.list_SP])
        
        # Initialize coefficient matrix and density matrix as Coe and Rho
        Coe = np.eye(Nind)
        Rho = np.zeros((Nind,Nind))
        
        # Compute initial Rho(0)
        for i_gam in range(Nind):
            for i_del in range(Nind):
                Rho[i_gam, i_del] = Decimal(sum(Coe[i_gam, i]*Coe[i_del, i] for i in range(self.A)))
                
        maxiter, epsl = 100, 1.0e-5
        diff, i_count = 1.0, 0
        
        oldE = np.zeros(Nind, dtype=object)
        newE = np.zeros(Nind, dtype=object)
        
        while i_count < maxiter and diff > epsl:
            MHF = np.zeros((Nind,Nind))
            
            # Make Hartree-Fock matrix MHF
            for i_alp in range(Nind):
                for i_bet in range(Nind):
                    if self.list_SP[i_alp][2:] != self.list_SP[i_bet][2:]: continue
                    sumTBME = sum(
                        Decimal(Rho[i_gam, i_del])* Decimal(self.twobody(i_alp,i_gam,i_bet,i_del)) # FIXME: need to change self.twobody
                        for i_gam in range(Nind)
                        for i_del in range(Nind)
                        if (self.list_SP[i_alp][4] + self.list_SP[i_gam][4] == self.list_SP[i_bet][4] + self.list_SP[i_del][4]) and 
                        (self.list_SP[i_alp][5]+self.list_SP[i_gam][5] == self.list_SP[i_bet][5]+self.list_SP[i_del][5])
                    )
                    MHF[i_alp, i_bet] = Decimal(sumTBME+(Decimal(SPH[i_alp]) if i_alp==i_bet else 0))
                    
            # Diagonalize and get the eigen states of MHF
            Eeig, Coe = np.linalg.eigh(np.array(MHF, dtype=float))
            Coe = np.array(Coe, dtype=object)
            
            # Update density matrix
            Rho = np.array([
                [sum(Decimal(Coe[i_alp,i])*Decimal(Coe[i_bet,i]) for i in range(self.A)) 
                for i_bet in range(Nind)]
                for i_alp in range(Nind)
            ])
            
            # Get new energies and calculate convergence
            newE = np.array([Decimal(e) for e in Eeig], dtype=object)
            diff = sum(abs(newE[i]-oldE[i])/Nind for i in range(Nind))
            oldE = newE
            i_count += 1
            
            # Print results
            print("Single-particle energies, ordering may have changed")
            for i in range(Nind):
                print('{0:4d} {1:.4f}'.format(i,Decimal(oldE[i])))
        
    # Function for generate one-body Hamiltonian, with energy gap 41*A^{-1/3}-25*A{-2/3} MeV
    def onebody(self, SP_State):                            
        e_gap = 41*np.power(self.A,-0.333)-21*np.power(self.A,-0.666)
        n, l = SP_State[1], SP_State[2]                     
        return e_gap*(2*n + l + 1.5)  
    
    # Function for two-body interaction, initial states are denoted as (beta, delta) and final states are denoted as (alpha, beta)...
    # Output of this function is expected to be two-body matrix elements (me2b)
    def twobody(self, alpha, gamma, beta, delta):           
        #print('Note: Daniel and Michael, we need to improve this part-Find the me2b and connect it between two states')
        return (random.randint(0,100)-50)/100               # Currently Assign a random value

    # Function for Kronecker Delta
    def krondelt(self, a, b):
        if a == b: return 1
        else: return 0

## Main 

Calculate the energy states from solving HF equation, but need to be refined in future (Nov 27 2024 comment)

In [13]:
if __name__ == '__main__':
    O16 = SP_States(12,6)
    print(O16.N)
    O16_SP_List = O16.generate_lists_SP()
    
    print(O16_SP_List)
    
    TB_O16 = TB_States(O16)
    A = TB_O16.generate_lists_TB()
    
    Hamiltonian(O16).generate_HFMatrix()

6
[[0, 0, 0, 0.5, -0.5, -0.5], [1, 0, 0, 0.5, 0.5, -0.5], [2, 0, 1, 1.5, -1.5, -0.5], [3, 0, 1, 1.5, -0.5, -0.5], [4, 0, 1, 1.5, 0.5, -0.5], [5, 0, 1, 1.5, 1.5, -0.5], [6, 0, 0, 0.5, -0.5, 0.5], [7, 0, 0, 0.5, 0.5, 0.5], [8, 0, 1, 1.5, -1.5, 0.5], [9, 0, 1, 1.5, -0.5, 0.5], [10, 0, 1, 1.5, 0.5, 0.5], [11, 0, 1, 1.5, 1.5, 0.5]]
Single-particle energies, ordering may have changed
   0 19.5351
   1 20.6151
   2 21.2651
   3 22.3651
   4 33.5752
   5 33.7452
   6 34.4652
   7 34.6652
   8 35.0452
   9 35.3252
  10 35.4552
  11 36.0452
Single-particle energies, ordering may have changed
   0 18.5051
   1 19.1851
   2 20.4851
   3 21.3151
   4 31.6852
   5 33.5052
   6 33.6852
   7 34.2052
   8 34.2852
   9 34.5552
  10 35.2552
  11 35.4552
Single-particle energies, ordering may have changed
   0 20.1051
   1 20.1151
   2 22.0551
   3 22.4151
   4 33.5752
   5 34.2852
   6 34.3752
   7 34.5352
   8 34.5652
   9 35.6852
  10 36.1252
  11 36.7452
Single-particle energies, ordering may have cha